In [1]:
import os
import ast
import shutil
import pandas  as pd
import numpy as np

In [2]:
df = pd.read_csv('bio_metadata.csv')

In [3]:
df.sample(12)

,href,language_num,sex,birth_place,native_language,other_languages,age_sex,age_of_english_onset,english_learning_method,english_residence,length_of_english_residence,age
892,http://accent.gmu.edu/browse_language.php?func...,english535,female,"['massachusetts,', 'usa']",english\n(eng),['none'],"['22,', 'female', '']",0.0,naturalistic,['usa'],22.0,22.0
132,http://accent.gmu.edu/browse_language.php?func...,mandarin133,female,"['beijing,', 'china']",mandarin\n(cmn),['none'],"['26,', 'female', '']",10.0,academic,['usa'],3.0,26.0
855,http://accent.gmu.edu/browse_language.php?func...,english498,male,"['columbia,', 'canada']",english\n(eng),"['french', 'german', '']","['27,', 'male', '']",0.0,naturalistic,['canada'],27.0,27.0
381,http://accent.gmu.edu/browse_language.php?func...,english24,male,"['scotland,', 'uk']",english\n(eng),"['french', '']","['37,', 'male', '']",0.0,naturalistic,"['uk,usa']",37.0,37.0
1204,http://accent.gmu.edu/browse_language.php?func...,italian3,male,"['florence,', 'italy']",italian\n(ita),"['french', 'japanese', '']","['55,', 'male', '']",21.0,academic,['usa'],34.0,55.0
1051,http://accent.gmu.edu/browse_language.php?func...,dutch34,female,"['bornem,', 'belgium']",dutch\n(nld),"['french', '']","['22,', 'female', '']",14.0,academic,[''],0.0,22.0
142,http://accent.gmu.edu/browse_language.php?func...,mandarin143,female,"['anhui,', 'china']",mandarin\n(cmn),['none'],"['25,', 'female', '']",10.0,academic,['usa'],0.2,25.0
1407,http://accent.gmu.edu/browse_language.php?func...,portuguese66,female,"['janeiro,', 'brazil']",portuguese\n(por),"['spanish', 'italian', '']","['24,', 'female', '']",8.0,academic,[''],0.0,24.0
32,http://accent.gmu.edu/browse_language.php?func...,mandarin33,male,"['beijing,', 'china']",mandarin\n(cmn),"['french', '']","['25,', 'male', '']",6.0,naturalistic,"['thailand,', 'australia,', 'usa']",20.0,25.0
149,http://accent.gmu.edu/browse_language.php?func...,mandarin150,female,"['shanxi,', 'china']",mandarin\n(cmn),"['spanish', 'german', '']","['18,', 'female', '']",12.0,academic,['nigeria'],4.0,18.0


In [4]:
df['english_residence'] = df['english_residence'].apply(ast.literal_eval)
# check if there are any of the entries in the english_residence column that have more than one in the list 
df['english_residence'].apply(lambda x: len(x)).value_counts()
# convert all entries in the english_residence column to lowercase
df['english_residence'] = df['english_residence'].apply(lambda x: [i.lower() for i in x])
# drop the rows where the english_residence column is empty or a number
df = df[~df['english_residence'].apply(lambda x: len(x)==0)]

In [5]:
# get all the unique values in the english_residence column
unique_residences = set()
df['english_residence'].apply(lambda x: unique_residences.update(x))
unique_residences = set([x for x in unique_residences if x.isalpha()])
# unique_residences

In [6]:
# there are some entries that english_residence that are like so [usa, uk, canada] which should be split into [usa], [uk], [canada]
df = df.explode('english_residence')
df.head()

,href,language_num,sex,birth_place,native_language,other_languages,age_sex,age_of_english_onset,english_learning_method,english_residence,length_of_english_residence,age
0,http://accent.gmu.edu/browse_language.php?func...,mandarin1,female,"['shanxi,', 'china']",mandarin\n(cmn),['none'],"['26,', 'female', '']",13.0,academic,usa,2.0,26.0
1,http://accent.gmu.edu/browse_language.php?func...,mandarin2,female,"['nanjing,', 'china']",mandarin\n(cmn),"['japanese', '']","['38,', 'female', '']",14.0,academic,usa,0.8,38.0
2,http://accent.gmu.edu/browse_language.php?func...,mandarin3,male,"['jilin,', 'china']",mandarin\n(cmn),"['italian', 'german', 'french', '']","['43,', 'male', '']",10.0,academic,usa,14.0,43.0
3,http://accent.gmu.edu/browse_language.php?func...,mandarin4,female,"['shanghai,', 'china']",mandarin\n(cmn),"['japanese', '']","['24,', 'female', '']",6.0,academic,usa,1.0,24.0
4,http://accent.gmu.edu/browse_language.php?func...,mandarin5,female,"['beijing,', 'china']",mandarin\n(cmn),['none'],"['31,', 'female', '']",12.0,academic,usa,2.0,31.0


In [7]:
# drop values like so '3.5'
df = df[~df['english_residence'].apply(lambda x: any(i.isdigit() for i in x))]

In [8]:
# drop empty strings like so ""
df = df[df['english_residence'].apply(lambda x: len(x)>0)]

In [9]:
# there are some like this usa,
# we will remove the comma
df['english_residence'] = df['english_residence'].apply(lambda x: x.replace(',', ''))

In [10]:
# new and zealand is one country
df['english_residence'] = df['english_residence'].apply(lambda x: 'new zealand' if x == 'new' else x)
df['english_residence'] = df['english_residence'].apply(lambda x: 'new zealand' if x == 'zealand' else x)

In [11]:
df['english_residence'].unique()

array(['usa', 'south', 'africa', 'singapore', 'uk', 'new zealand',
       'canada', 'australia', 'thailand', 'northern', 'mariana',
       'islands', 'hong', 'kong', 'nigeria', 'ukusa', 'jamaicausa',
       'australiausa', 'trinidadusa', 'papua', 'guineaukusa', 'ireland',
       'ukcanada', 'jamaica', 'india', 'us', 'virgin', 'wales',
       'singaporeusa', 'philippinescanada', 'guyana', 'fiji', 'antigua',
       'and', 'barbuda', 'barbados', 'scotland', 'sierra', 'leone',
       'isle', 'of', 'man', 'belize', 'liberia', 'philippines', 'the',
       'bahamas', 'ghana', 'panama', 'cayman', 'ausstralia', 'cameroon',
       'usq', 'zimbabwe', 'italy', 'ukaustralia', 'island', 'brazil'],
      dtype=object)

In [12]:
# get the count of each residence
df['english_residence'].value_counts()

english_residence
usa                  1031
uk                    115
canada                 78
australia              69
new zealand            31
ireland                19
singapore              10
africa                  9
south                   9
scotland                5
jamaica                 5
ukusa                   4
islands                 3
guyana                  2
philippines             2
kong                    2
nigeria                 2
hong                    2
mariana                 2
northern                2
of                      1
man                     1
belize                  1
liberia                 1
bahamas                 1
the                     1
usq                     1
ghana                   1
panama                  1
cayman                  1
ausstralia              1
cameroon                1
leone                   1
zimbabwe                1
italy                   1
ukaustralia             1
island                  1
isle                

In [13]:
# drop all with value count less than 2
df = df[df['english_residence'].map(df['english_residence'].value_counts()) > 2]
df['english_residence'].value_counts()

english_residence
usa            1031
uk              115
canada           78
australia        69
new zealand      31
ireland          19
singapore        10
south             9
africa            9
jamaica           5
scotland          5
ukusa             4
islands           3
Name: count, dtype: int64

In [14]:
np.array(df['english_residence'].unique())

array(['usa', 'south', 'africa', 'singapore', 'uk', 'new zealand',
       'canada', 'australia', 'islands', 'ukusa', 'ireland', 'jamaica',
       'scotland'], dtype=object)

In [15]:
# drop ukusa
df = df[df['english_residence'] != 'ukusa']
df['english_residence'].value_counts()

english_residence
usa            1031
uk              115
canada           78
australia        69
new zealand      31
ireland          19
singapore        10
south             9
africa            9
jamaica           5
scotland          5
islands           3
Name: count, dtype: int64

In [16]:
# seperate usa, uk and canada into a separate dataframe
usa = df[df['english_residence'] == 'usa']
uk = df[df['english_residence'] == 'uk']
canada = df[df['english_residence'] == 'canada']

# combine the three dataframes
usa_uk_canada = pd.concat([usa, uk, canada])

# drop the three countries from the original dataframe
df = df[~df['english_residence'].isin(['usa', 'uk', 'canada'])]

# value counts of the english_residence in the new dataframe
usa_uk_canada['english_residence'].value_counts()


english_residence
usa       1031
uk         115
canada      78
Name: count, dtype: int64

In [17]:
df['english_residence'].value_counts()

english_residence
australia      69
new zealand    31
ireland        19
singapore      10
south           9
africa          9
jamaica         5
scotland        5
islands         3
Name: count, dtype: int64

In [18]:
# save the dataframes to csv
df.to_csv('non_native_bio_metadata.csv', index=False)
usa_uk_canada.to_csv('native_bio_metadata.csv', index=False)

In [19]:
# load the native and non native dataframes
native_df = pd.read_csv('native_bio_metadata.csv')
non_native_df = pd.read_csv('non_native_bio_metadata.csv')



In [20]:
# show the first 5 rows of the native dataframe
native_df.head()

,href,language_num,sex,birth_place,native_language,other_languages,age_sex,age_of_english_onset,english_learning_method,english_residence,length_of_english_residence,age
0,http://accent.gmu.edu/browse_language.php?func...,mandarin1,female,"['shanxi,', 'china']",mandarin\n(cmn),['none'],"['26,', 'female', '']",13.0,academic,usa,2.0,26.0
1,http://accent.gmu.edu/browse_language.php?func...,mandarin2,female,"['nanjing,', 'china']",mandarin\n(cmn),"['japanese', '']","['38,', 'female', '']",14.0,academic,usa,0.8,38.0
2,http://accent.gmu.edu/browse_language.php?func...,mandarin3,male,"['jilin,', 'china']",mandarin\n(cmn),"['italian', 'german', 'french', '']","['43,', 'male', '']",10.0,academic,usa,14.0,43.0
3,http://accent.gmu.edu/browse_language.php?func...,mandarin4,female,"['shanghai,', 'china']",mandarin\n(cmn),"['japanese', '']","['24,', 'female', '']",6.0,academic,usa,1.0,24.0
4,http://accent.gmu.edu/browse_language.php?func...,mandarin5,female,"['beijing,', 'china']",mandarin\n(cmn),['none'],"['31,', 'female', '']",12.0,academic,usa,2.0,31.0


In [21]:
# show the first 5 rows of the non native dataframe
non_native_df.head()

,href,language_num,sex,birth_place,native_language,other_languages,age_sex,age_of_english_onset,english_learning_method,english_residence,length_of_english_residence,age
0,http://accent.gmu.edu/browse_language.php?func...,mandarin10,male,"['beijing,', 'china']",mandarin\n(cmn),['none'],"['19,', 'male', '']",3.0,academic,south,3.75,19.0
1,http://accent.gmu.edu/browse_language.php?func...,mandarin10,male,"['beijing,', 'china']",mandarin\n(cmn),['none'],"['19,', 'male', '']",3.0,academic,africa,3.75,19.0
2,http://accent.gmu.edu/browse_language.php?func...,mandarin12,male,"['singapore,', 'singapore']",mandarin\n(cmn),"['cantonese', 'spanish', 'french', '']","['23,', 'male', '']",1.0,naturalistic,singapore,23.00,23.0
3,http://accent.gmu.edu/browse_language.php?func...,mandarin15,female,"['tainan,', 'taiwan']",mandarin\n(cmn),"['french', 'swedish', 'japanese', '']","['28,', 'female', '']",11.0,academic,new zealand,4.00,28.0
4,http://accent.gmu.edu/browse_language.php?func...,mandarin15,female,"['tainan,', 'taiwan']",mandarin\n(cmn),"['french', 'swedish', 'japanese', '']","['28,', 'female', '']",11.0,academic,new zealand,4.00,28.0


In [22]:
# create the folders
os.makedirs('data/native', exist_ok=True)
os.makedirs('data/non_native', exist_ok=True)

In [23]:
# all the audio files are now in the data/audio folder, seperate them using the language_num column
for i, row in native_df.iterrows():
    shutil.copy(f'data/audio/{row["language_num"]}.wav', f'data/native/{row["language_num"]}.wav')

In [24]:
for i, row in non_native_df.iterrows():
    shutil.copy(f'data/audio/{row["language_num"]}.wav', f'data/non_native/{row["language_num"]}.wav')